In [25]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving Corpas.csv to Corpas.csv


In [14]:
!pip install transformers

In [26]:
pip install simpletransformers==0.50.0

In [27]:
import pandas as pd
df = pd.read_csv('Corpas.csv')
df.head()

,text,category
0,ভূমিকা>বাংলা সাহিত্যের প্রথম সার্থক উপন্যাস স্...,Shadhu
1,অষ্টাদশ শতাব্দীতে ব্রিটিশ বাণিজ্যপুঁজির প্রভা...,Shadhu
2,ওপনিবেশিক সেই শাসনপর্বে মধ্যযুগীয় খ্রামগুচ্...,Shadhu
3,বাংলা সাহিত্যে এর আগে উপন্যাসের লক্ষণাক্রান্...,Shadhu
4,পাশ্চাত্য শিক্ষায়প্যাচার নকৃশা বা কাদন্বরীর...,Shadhu


In [18]:
del df['title']
df.head()
# del df['id']
# df.head()
# del df['date']
# df.head()
# del df['article']
# df.head()

,category,text
0,ব্যান্ড,এবার তোমার কথাগুলোর\nকাছে আমি অপরাজিত\nজমে থাক...
1,দেশাত্মবোধক গান,আমার ভায়ের রক্তে রাঙানো একুশে ফেব্রুয়ারি\nআমি ...
2,বাউল,আমি কুকুর হইয়া দেই পাহাড়া\nদয়াল চান্দের ঘড় বাড়...
3,বাউল,তুমি এসমে আজ যত তরানে ওলা\nগাউছেলাম বাবা নুরে ...
4,নজরুল গীতি,আমার আপনার চেয়ে আপন যে জন\nখুঁজি তারে আমি আপন...


In [28]:
labels = list(set(df['category']))
print(labels)

['Shadhu']


In [29]:
for i in range(len(labels)):
  df.loc[df['category'] == labels[i], 'category'] = i
df = df.sample(frac=1) # randomize data
df.head(20)

,text,category
61766,আমি টানতে শুরু করিলাম,0
73661,নিরাশ্রয় নিঃসন্বল পথে পথে সে ভিক্ষা করিয়...,0
79152,চক্ষুতেতার এক বিন্দু জল চক চক করিতে লাগিল,0
63270,যেমন বিস্মিত হইলাম তেমিন বিরক্ত হইলাম,0
59124,কমল কহিল হতো না,0
79312,স্বায়ী বাহিরে পড়িয়া রহিলেন,0
76681,মালতী ক্রোড়ের উপর মাথা নাড়িয়া জানাইল যে ত...,0
78097,একজন এক এক পদগায় আর একজন দ্বিতীয় পদ গায়,0
13081,তাহার টাকাকড়ি বিষয় সম্পত্তিতে অকস্মাৎ এতবড...,0
35130,আমারও প্রাণ ফিরিয়ে দাও স্বামী ত্যাগ করো,0


In [7]:
df.dtypes

text        object
category    object
dtype: object

In [30]:
# remove float type text rows
df = df.convert_dtypes(convert_string=True).dropna()

In [31]:
list(set(df['category']))

[0]

In [32]:
# split train test
from sklearn.model_selection import train_test_split

train, eval = train_test_split(df,test_size=0.2)
print(len(train))
print(len(eval))

64543
16136


In [33]:
from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "sagorsarker/bangla-bert-base", args={"num_train_epoch": 1, "reprocess_input_data": False, "overwrite_output_dir": True}
)

# Train the model


model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval)

# predictions, raw_outputs = model.predict(["Some arbitary sentence"])
print(result)

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/64543 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/8068 [00:00<?, ?it/s]

Saving vocabulary to outputs/checkpoint-2000/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to outputs/checkpoint-2000/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to outputs/checkpoint-2000/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to outputs/checkpoint-2000/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to outputs/checkpoint-2000/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to outputs/checkpoint-2000/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to outputs/checkpoint-2000/vocab.txt: vocabulary indices are not consecutive. Please check that the 

  0%|          | 0/16136 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4492fc7680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Running Evaluation:   0%|          | 0/2017 [00:00<?, ?it/s]

{'mcc': 0.0, 'tp': 0, 'tn': 16136, 'fp': 0, 'fn': 0, 'eval_loss': 4.3738638935079653e-07}


In [34]:
import numpy

perplexity = numpy.exp(4.3738638935079653e-07)


In [35]:
perplexity

1.000000437386485

In [36]:
from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "sagorsarker/bangla-bert-base", args={"num_train_epoch": 1, "reprocess_input_data": False, "overwrite_output_dir": True}
)

# Train the model


# model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval)

# predictions, raw_outputs = model.predict(["Some arbitary sentence"])
print(result)

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Running Evaluation:   0%|          | 0/2017 [00:00<?, ?it/s]

{'mcc': 0.0, 'tp': 0, 'tn': 9903, 'fp': 6233, 'fn': 0, 'eval_loss': 0.6731389149262519}


In [37]:
import numpy

perplexity = numpy.exp(0.6731389149262519)
perplexity

1.9603811427541176